In [13]:
import pandas as pd

data = pd.read_excel('data.xlsx')
data.head()

,Name of the NBS intervention,Native title of the NBS intervention,City,Country,City population,Duration,Present stage of the intervention,Spatial scale,NBS area,Type of area before implementation of the NBS,...,Economic impacts,Social and cultural impacts,Type of reported impacts,Presence of formal monitoring system,Presence of indicators used in reporting,Presence of monitoring/evaluation reports,Availability of a web-based monitoring tool,List of references,Last updated,Link
0,River rehabilitation and creation of green cor...,Porsuk Çayı Kıyısı Yeşil Koridor,Eskişehir,Turkey,887475.0,2003 - 2009,Completed,"Meso-scale: Regional, metropolitan and urban l...",257000m²,"Natural Heritage Area/Untouched nature, Public...",...,Attraction of business and investment,Safety\n- Improved community safety to climate...,Achieved impacts,Unknown,No evidence in public records,No evidence in public records,No evidence in public records,"1) Arslantas, F., Sanalan, K.C. Cil, A. (2020)...","Sat, 10/23/2021 - 20:54",https://una.city/nbs/eskisehir/river-rehabilit...
1,Tirana Orbital Forest,Tirana Orbital Forest,Tirana (FUA),Albania,980000.0,2017 - ongoing,Ongoing,"Meso-scale: Regional, metropolitan and urban l...",140000000m²,"Agricultural area or farmland, Public Greenspa...",...,Unknown,Social justice and cohesion\n- Improved liveab...,"Expected impacts\n, Achieved impacts",Unknown,Yes,No evidence in public records,No evidence in public records,1) Sustainable Cities Platform. (n.a.). Orbita...,"Sat, 10/23/2021 - 20:54",https://una.city/nbs/tirana/tirana-orbital-forest
2,Air quality project at Hunters Bar Infant Scho...,NaN,Sheffield,United Kingdom,548261.0,2018 - 2022,Ongoing,Sub-microscale: Street scale (including buildi...,3600m²,"Residential, Building",...,Unknown,Health and wellbeing\n- Improved physical heal...,Expected impacts,Unknown,No evidence in public records,No evidence in public records,No evidence in public records,"(1) University of Sheffield (no date), BREATHE...","Sat, 10/23/2021 - 20:54",https://una.city/nbs/sheffield/air-quality-pro...
3,Venice-Mestre Angel Hospital,Ospedale Dell'Angelo Mestre,Venezia,Italy,259789.0,2004 - 2008,Completed,Sub-microscale: Street scale (including buildi...,117600m²,Previous derelict area,...,Unknown,Health and wellbeing\n- Improved mental health...,Achieved impacts,Unknown,No evidence in public records,No evidence in public records,No evidence in public records,Ref. 1. Green Roofs. com (2018) VENICE-MESTRE...,"Sat, 10/23/2021 - 20:54",https://una.city/nbs/venezia/venice-mestre-ang...
4,Climate Adaptation Strategy,NaN,Munich,Germany,1346481.0,2017 - ongoing,Ongoing,"Meso-scale: Regional, metropolitan and urban l...",NaN,Other,...,Unknown,Social justice and cohesion\n- Improved liveab...,Expected impacts,Unknown,Yes,No evidence in public records,No evidence in public records,"Ref 1. Kiss, B., Wamsler, C. (2018) NATURVATIO...","Sat, 10/23/2021 - 20:54",https://una.city/nbs/munich/climate-adaptation...


In [14]:
# Split duration into two columns
# TODO: 'unknown', 'ongoing', 'pre-1990', 'in planning phase'

cleaned_data = data.copy()
cleaned_data[['Starting year', 'Finish year']] = cleaned_data['Duration'].str.split(' - ', expand=True)
# cleaned_data['Starting year'] = cleaned_data['Starting year'].astype('Int64')
# cleaned_data['Finish year']   = cleaned_data['Finish year'].astype('Int64')
cleaned_data = cleaned_data.drop(columns=['Duration'])

In [15]:
# Set all columns with a single selection value to a category type

category_columns = [
    'Present stage of the intervention', 
    'Spatial scale', 
    'Governance arrangements', 
    'NBS intervention implemented in response to a national regulations/strategy/plan',
    'NBS intervention implemented in response to a local regulation/strategy/plan', 
    'NBS intervention implemented in response to an EU Directive/Strategy', 
    'Total cost', 
    'Presence of formal monitoring system',  
    'Presence of indicators used in reporting',
    'Presence of monitoring/evaluation reports',
    'Availability of a web-based monitoring tool'
]

cleaned_data[category_columns] = cleaned_data[category_columns].astype('category')

In [16]:
# Set al numerical columns as the right type

cleaned_data['NBS area'] = cleaned_data['NBS area'].astype(str).str.extract(r'(\d+\.?\d*)')[0].astype('Int64')
cleaned_data['City population'] = cleaned_data['City population'].astype('Int64')

In [17]:
# Transfer column with multiple selection into multiple columns with binary values if option is applicable

cleaned_data['Type of area before implementation of the NBS'] = (
    cleaned_data['Type of area before implementation of the NBS']
    .fillna('')
    .astype(str)
    .str.split(',')
)

cleaned_data['Type of area before implementation of the NBS'] = cleaned_data['Type of area before implementation of the NBS'].apply(
    lambda lst: [x.strip() for x in lst if x.strip() != ""]
)

one_hot = cleaned_data['Type of area before implementation of the NBS'].str.join('|').str.get_dummies()
one_hot = one_hot.add_prefix(f"{'Type of area before implementation of NBS'}")

cleaned_data = cleaned_data.join(one_hot)
cleaned_data = cleaned_data.drop(columns='Type of area before implementation of the NBS')

cleaned_data.head()

,Name of the NBS intervention,Native title of the NBS intervention,City,Country,City population,Present stage of the intervention,Spatial scale,NBS area,Short description of the intervention,Type of nature-based solution/Ecological domain,...,Type of area before implementation of NBSCentral Business District / City Centre,Type of area before implementation of NBSCultural Heritage Area,Type of area before implementation of NBSIndustrial,Type of area before implementation of NBSNatural Heritage Area/Untouched nature,Type of area before implementation of NBSOther,Type of area before implementation of NBSPrevious derelict area,Type of area before implementation of NBSProtected Area,Type of area before implementation of NBSPublic Greenspace Area,Type of area before implementation of NBSResidential,Type of area before implementation of NBSUnknown
0,River rehabilitation and creation of green cor...,Porsuk Çayı Kıyısı Yeşil Koridor,Eskişehir,Turkey,887475,Completed,"Meso-scale: Regional, metropolitan and urban l...",257000,The Porsuk Stream divides the city of Eskişehi...,Grey infrastructure featuring greens\nBlue inf...,...,0,0,0,1,0,0,0,1,0,0
1,Tirana Orbital Forest,Tirana Orbital Forest,Tirana (FUA),Albania,980000,Ongoing,"Meso-scale: Regional, metropolitan and urban l...",140000000,The Orbital Forest of Tirana is an innovative ...,Parks and urban forests\n- Large urban parks o...,...,0,0,0,0,0,0,0,1,0,0
2,Air quality project at Hunters Bar Infant Scho...,NaN,Sheffield,United Kingdom,548261,Ongoing,Sub-microscale: Street scale (including buildi...,3600,This project started as a grassroots movement ...,Nature on buildings (external)\n- Green walls ...,...,0,0,0,0,0,0,0,0,1,0
3,Venice-Mestre Angel Hospital,Ospedale Dell'Angelo Mestre,Venezia,Italy,259789,Completed,Sub-microscale: Street scale (including buildi...,117600,The campus of the Venice-Mestre Hospital (Ospe...,Nature on buildings (external)\n- Green roofs\...,...,0,0,0,0,0,1,0,0,0,0
4,Climate Adaptation Strategy,NaN,Munich,Germany,1346481,Ongoing,"Meso-scale: Regional, metropolitan and urban l...",<NA>,The Climate Adaptation Strategy (CAS) was appr...,Nature on buildings (external)\n- Green roofs\...,...,0,0,0,0,1,0,0,0,0,0


In [18]:
# A function to transfer a column with multiple selection into multiple columns with binary values if option is applicable

def one_hot_multilabel(df, col):
    df[col] = (
        df[col]
        .fillna('')
        .astype(str)
        .str.split(',')
        .apply(lambda lst: [x.strip() for x in lst if x.strip() != ""])
    )

    one_hot = df[col].str.join('|').str.get_dummies()
    one_hot = one_hot.add_prefix(f"{col}: ")

    df = df.join(one_hot).drop(columns=[col])

    return df

In [ ]:
# TODO: function does not work currently, since columns have different ways of writing down the options
 
'''
columns_multiple_selection = [
    'Type of area before implementation of the NBS',
    'Type of nature-based solution/Ecological domain',
    'Sustainability challenge(s) addressed',
    'Focus of the project',
    'Climate change adaptation: What activities are implemented to realize the conservation goals and targets?',
    'Climate change mitigation: What activities are implemented to realize the conservation goals and targets?',
    'Habitats and biodiversity conservation: What activities are implemented to realize the conservation goals and targets?',
    'Habitats and biodiversity restoration: What activities are implemented to realize the restoration goals and targets?',
    'Primary Beneficiaries',
    'Key actors - initiating organization',
    'Participatory methods/forms of community involvement used',
    'Source(s) of funding',
    'Type of fund(s) used',
    'Type of non-financial contribution',
    'Who provided the non-financial contribution?',
    'Environmental impacts',
    'Economic impacts',
    'Social and cultural impacts',
    'Type of reported impacts'
]

for col in columns_multiple_selection:
    cleaned_data = one_hot_multilabel(cleaned_data, col)
'''

"columns_multiple_selection = [\n    'Type of area before implementation of the NBS',\n    'Type of nature-based solution/Ecological domain',\n    'Sustainability challenge(s) addressed',\n    'Focus of the project',\n    'Climate change adaptation: What activities are implemented to realize the conservation goals and targets?',\n    'Climate change mitigation: What activities are implemented to realize the conservation goals and targets?',\n    'Habitats and biodiversity conservation: What activities are implemented to realize the conservation goals and targets?',\n    'Habitats and biodiversity restoration: What activities are implemented to realize the restoration goals and targets?',\n    'Primary Beneficiaries',\n    'Key actors - initiating organization',\n    'Participatory methods/forms of community involvement used',\n    'Source(s) of funding',\n    'Type of fund(s) used',\n    'Type of non-financial contribution',\n    'Who provided the non-financial contribution?',\n    'En

In [24]:
# Change value of the column 'No evidence in public records' or 'Unknown' to 'No'

columns = ['Presence of indicators used in reporting', 'Presence of monitoring/evaluation reports', 'Availability of a web-based monitoring tool', 'Presence of formal monitoring system']

cleaned_data[columns] = \
    cleaned_data[columns].replace({
        'No evidence in public records' : 0,
        'Unknown' : 0,
        'No' : 0,
        'Yes' : 1
    })

In [25]:
cleaned_data.head()

,Name of the NBS intervention,Native title of the NBS intervention,City,Country,City population,Present stage of the intervention,Spatial scale,NBS area,Short description of the intervention,Type of nature-based solution/Ecological domain,...,Type of area before implementation of NBSCentral Business District / City Centre,Type of area before implementation of NBSCultural Heritage Area,Type of area before implementation of NBSIndustrial,Type of area before implementation of NBSNatural Heritage Area/Untouched nature,Type of area before implementation of NBSOther,Type of area before implementation of NBSPrevious derelict area,Type of area before implementation of NBSProtected Area,Type of area before implementation of NBSPublic Greenspace Area,Type of area before implementation of NBSResidential,Type of area before implementation of NBSUnknown
0,River rehabilitation and creation of green cor...,Porsuk Çayı Kıyısı Yeşil Koridor,Eskişehir,Turkey,887475,Completed,"Meso-scale: Regional, metropolitan and urban l...",257000,The Porsuk Stream divides the city of Eskişehi...,Grey infrastructure featuring greens\nBlue inf...,...,0,0,0,1,0,0,0,1,0,0
1,Tirana Orbital Forest,Tirana Orbital Forest,Tirana (FUA),Albania,980000,Ongoing,"Meso-scale: Regional, metropolitan and urban l...",140000000,The Orbital Forest of Tirana is an innovative ...,Parks and urban forests\n- Large urban parks o...,...,0,0,0,0,0,0,0,1,0,0
2,Air quality project at Hunters Bar Infant Scho...,NaN,Sheffield,United Kingdom,548261,Ongoing,Sub-microscale: Street scale (including buildi...,3600,This project started as a grassroots movement ...,Nature on buildings (external)\n- Green walls ...,...,0,0,0,0,0,0,0,0,1,0
3,Venice-Mestre Angel Hospital,Ospedale Dell'Angelo Mestre,Venezia,Italy,259789,Completed,Sub-microscale: Street scale (including buildi...,117600,The campus of the Venice-Mestre Hospital (Ospe...,Nature on buildings (external)\n- Green roofs\...,...,0,0,0,0,0,1,0,0,0,0
4,Climate Adaptation Strategy,NaN,Munich,Germany,1346481,Ongoing,"Meso-scale: Regional, metropolitan and urban l...",<NA>,The Climate Adaptation Strategy (CAS) was appr...,Nature on buildings (external)\n- Green roofs\...,...,0,0,0,0,1,0,0,0,0,0
